#
Colab Setup:
1. Create a new Colab notebook.
2. Install necessary Python packages:



In [ ]:
!pip install flask flask-cors pyngrok google-generativeai

## Import libraries and configure your LLM API key (using Colab's Secrets manager is recommended for keys):

In [ ]:
import google.generativeai as genai
from google.colab import userdata # For secrets
import json
# Assuming you've stored your API key as 'GEMINI_API_KEY' in Colab secrets
try:
    api_key = userdata.get('GEMINI_API_KEY')
    genai.configure(api_key=api_key)
    print("Gemini API Key configured.")
except userdata.SecretNotFoundError:
    print("ERROR: Secret 'GEMINI_API_KEY' not found. Please add it in Colab's Secrets manager (key icon on the left).")
except Exception as e:
    print(f"An error occurred during API key configuration: {e}")

# Placeholder for the model
llm_model = None
if 'api_key' in locals() and api_key:
     try:
         llm_model = genai.GenerativeModel('gemini-1.5-flash') # Or another suitable model
         print("Gemini model initialized.")
     except Exception as e:
         print(f"Error initializing Gemini model: {e}")
else:
    print("Skipping model initialization due to missing API key.")

#Setup Imports and Run Flask Server

## setup imports

In [ ]:
from flask import Flask, request, jsonify, send_from_directory
from flask_cors import CORS
import threading
import os
from pyngrok import ngrok # Import ngrok
import re # For parsing LLM responses
!mkdir -p game_files

## Flask App Setup

In [ ]:
app = Flask(__name__, static_folder = 'game_files')
CORS(app)

## Game Logic and placeholder

## LLM Interaction Function

In [ ]:
'''
def get_llm_decision(game_state, available_actions, user_instruction):
    if not llm_model:
         return {"error": "LLM model not initialized."}

    # --- Crucial: Prompt Engineering ---
    # Provide context, rules, state, available actions, and the user's goal.
    # Ask for the output in a specific, parseable format (like JSON).
    prompt = f"""You are playing a game. Respond with ONLY the chosen action in the format specified.

    Game: Tic-Tac-Toe
    Your Mark: O (You are playing as O)
    Player Mark: X

    Current Board State:
    {json.dumps(game_state['board'], indent=2)}
    (Empty strings "" represent empty cells)

    Available Actions (Choose one by coordinates row, col):
    {json.dumps(available_actions, indent=2)}

    User Instruction: "{user_instruction}"

    Game Status: {game_state['status']}
    Current Turn: {game_state['turn']}

    Based on the user instruction and the current state, choose the best action from the available actions for player 'O'.
    Respond ONLY with the chosen action as a JSON object like this: {{"action": "place O at (row, col)"}} or like this if you need coordinates: {{"row": r, "col": c}}
    Example response for placing O in the top-left: {{"row": 0, "col": 0}}
    """

    print("--- Sending Prompt to LLM ---")
    # print(prompt) # Uncomment to debug the prompt

    try:
        response = llm_model.generate_content(prompt)
        print("--- Received Response from LLM ---")
        print(response.text)

        # --- Parse the LLM Response ---
        # This is critical and might need robust error handling
        try:
            # Try parsing directly if LLM gives perfect JSON (unlikely)
            # action_data = json.loads(response.text.strip())

            # More robust: Search for JSON-like structure or extract coords
            # Example naive extraction (adapt based on LLM output format):
            import re
            match = re.search(r'{\s*"row":\s*(\d+),\s*"col":\s*(\d+)\s*}', response.text)
            if match:
                row, col = int(match.group(1)), int(match.group(2))
                # Validate if this action is actually in available_actions
                if {"row": row, "col": col} in available_actions:
                    print(f"LLM chose valid action: row={row}, col={col}")
                    return {"row": row, "col": col}
                else:
                     print(f"LLM chose an INVALID action ({row},{col}), not in available actions.")
                     return {"error": f"LLM chose an invalid action: ({row},{col})"}

            # Fallback or different parsing if needed
            return {"error": "Could not parse valid action from LLM response.", "raw_response": response.text}

        except json.JSONDecodeError:
             print(f"LLM response was not valid JSON: {response.text}")
             return {"error": "LLM response was not valid JSON.", "raw_response": response.text}
        except Exception as e:
             print(f"Error parsing LLM response: {e}")
             return {"error": f"Error parsing LLM response: {e}", "raw_response": response.text}

    except Exception as e:
        print(f"Error calling LLM API: {e}")
        return {"error": f"Error calling LLM API: {e}"}
        '''

'\ndef get_llm_decision(game_state, available_actions, user_instruction):\n    if not llm_model:\n         return {"error": "LLM model not initialized."}\n\n    # --- Crucial: Prompt Engineering ---\n    # Provide context, rules, state, available actions, and the user\'s goal.\n    # Ask for the output in a specific, parseable format (like JSON).\n    prompt = f"""You are playing a game. Respond with ONLY the chosen action in the format specified.\n\n    Game: Tic-Tac-Toe\n    Your Mark: O (You are playing as O)\n    Player Mark: X\n\n    Current Board State:\n    {json.dumps(game_state[\'board\'], indent=2)}\n    (Empty strings "" represent empty cells)\n\n    Available Actions (Choose one by coordinates row, col):\n    {json.dumps(available_actions, indent=2)}\n\n    User Instruction: "{user_instruction}"\n\n    Game Status: {game_state[\'status\']}\n    Current Turn: {game_state[\'turn\']}\n\n    Based on the user instruction and the current state, choose the best action from the av

In [ ]:
# server.py or llm_controller.py

from flask import Flask, request, jsonify
import json
import re

app = Flask(__name__)

# Mockup LLM object
llm_model = ...  # Your LLM API object

@app.route("/llm/move", methods=["POST"])
def llm_move():
    data = request.json
    if not llm_model:
        return jsonify({"error": "LLM model not initialized."})

    units = data.get("units", [])
    instruction = data.get("instruction", "")

    prompt = f"""
You are playing a strategy game. Your goal is to move units based on the user's instruction.

Units:
{json.dumps(units, indent=2)}

Each unit has: unitId, x, y, and fuel.

User Instruction:
\"\"\"{instruction}\"\"\"

Respond with a list of move commands. Each command should look like:
{{"unitId": <number>, "x": <number>, "y": <number>, "queue": <true/false>}}

Only return the list of JSON objects. Example:
[
  {{"unitId": 1, "x": 3, "y": 4, "queue": false}},
  {{"unitId": 2, "x": 5, "y": 6, "queue": true}}
]
    """

    print("--- LLM Prompt ---")
    print(prompt)

    try:
        response = llm_model.generate_content(prompt)
        raw = response.text.strip()

        print("--- LLM Response ---")
        print(raw)

        try:
            move_plan = json.loads(raw)
            if isinstance(move_plan, list):
                return jsonify({"plan": move_plan})
            else:
                return jsonify({"error": "LLM did not return a list", "raw": raw})
        except Exception as e:
            return jsonify({"error": "Could not parse LLM output", "details": str(e), "raw": raw})

    except Exception as e:
        return jsonify({"error": f"Error calling LLM API: {e}"})


## API Endpoint

In [ ]:
@app.route('/api/llm_move', methods=['POST'])
def handle_llm_move():
    data = request.json
    game_state = data.get('gameState')
    available_actions = data.get('availableActions') # JS needs to calculate these!
    user_instruction = data.get('instruction')

    if not game_state or not available_actions or user_instruction is None:
        return jsonify({"error": "Missing gameState, availableActions, or instruction"}), 400

    # Basic validation (more needed in a real app)
    if game_state.get('currentPlayer') != 'O': # Assuming LLM plays 'O'
         return jsonify({"error": "Not LLM's turn"}), 400
    if game_state.get('status') != 'playing':
         return jsonify({"error": "Game is not active"}), 400

    decision = get_llm_decision(game_state, available_actions, user_instruction)
    return jsonify(decision)

## Static File Serving

In [ ]:
# --- Static File Serving ---
# Create a directory in Colab's file system to hold your game files
if not os.path.exists('game_files'):
    os.makedirs('game_files')

# You need to UPLOAD your index.html, game.js, phaser.min.js, etc.
# into this 'game_files' directory using Colab's file browser (folder icon on left).

@app.route('/')
def index():
    # Serves index.html from the 'game_files' directory
    return send_from_directory('game_files', 'index.html')

@app.route('/<path:filename>')
def serve_static(filename):
    # Serves other files (game.js, phaser.min.js, assets)
    return send_from_directory('game_files', filename)


## Function to Run Flask App

In [ ]:
def run_flask():
  # Needs to run on port 8080 for some Colab environments, or choose another
  # Ngrok will tunnel to this port
  print("Starting Flask server...")
  app.run(host='0.0.0.0', port=8880) # Run on all interfaces, port 8080

## Start ngrok Tunnel

In [ ]:
def start_ngrok():
    try:
        # Terminate existing tunnels if any
        ngrok.kill()
        # Get ngrok auth token from Colab secrets if you have one (recommended for stable URLs)
        try:
             ngrok_auth_token = userdata.get('NGROK_AUTH_TOKEN')
             ngrok.set_auth_token(ngrok_auth_token)
             print("Ngrok auth token set.")
        except userdata.SecretNotFoundError:
             print("INFO: NGROK_AUTH_TOKEN secret not found. Using ngrok without auth token (temporary URL).")
        except Exception as e:
             print(f"Error setting ngrok auth token: {e}")

        # Start an HTTP tunnel on the same port Flask is running on
        public_url = ngrok.connect(8880, "http")
        print(f" * ngrok tunnel available at: {public_url}")
        return public_url
    except Exception as e:
        print(f"Error starting ngrok: {e}")
        return None

## Main Execution Logic

In [ ]:

public_url = start_ngrok()

print("\n--- Server Setup Complete ---")
if public_url:
    print(f"Access your game POC at: {public_url}")
    print(f"LLM endpoint will be at: {public_url}/api/llm_move") # Print full URL
else:
    print("Failed to start ngrok tunnel. Server might be running but not accessible publicly.")
    # Maybe exit or raise error if ngrok fails?

print("Make sure you have uploaded your index.html, game.js, and phaser.min.js into the 'game_files' directory in Colab.")


# --- Run Flask in Foreground (BLOCKING) ---
# This will block the cell from 'finishing' until you manually interrupt it (e.g., Runtime -> Interrupt execution).
print("\n--- Starting Flask Server (Foreground - Blocking) ---")
print(f"Flask is now running and listening on port 8880.")
print("Logs should appear directly below as requests come in.")
print("!!! The Colab cell will appear 'busy'. To stop the server, you MUST interrupt the kernel !!!")
try:
    # Run Flask on 0.0.0.0 to accept connections from ngrok
    app.run(host='0.0.0.0', port=8880, debug=False) # Turn debug=True for more Flask logs if needed, but can be verbose
except KeyboardInterrupt:
    print("\n--- Flask server stopped (KeyboardInterrupt) ---")

# The code below this line will NOT execute until the server is stopped.
print("--- Flask server has been shut down ---")


'''
# Start Flask in a separate thread so it doesn't block Colab execution
flask_thread = threading.Thread(target=run_flask, daemon=True)
flask_thread.start()

# Start ngrok and print the public URL
public_url = start_ngrok()

print("\n--- Server Setup Complete ---")
if public_url:
    print(f"Access your game POC at: {public_url}")
else:
    print("Failed to start ngrok tunnel. Server might be running but not accessible publicly.")
print("Flask server is running in the background.")
print("LLM endpoint is available at /api/llm_move (relative to the ngrok URL)")
print("Make sure you have uploaded your index.html, game.js, and phaser.min.js into the 'game_files' directory in Colab.")
# Keep the Colab cell running..
'''

Error setting ngrok auth token: Requesting secret NGROK_AUTH_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.


ERROR:pyngrok.process.ngrok:t=2025-06-10T03:14:56+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-06-10T03:14:56+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-06-10T03:14:56+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your aut

Error starting ngrok: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.

--- Server Setup Complete ---
Failed to start ngrok tunnel. Server might be running but not accessible publicly.
Make sure you have uploaded your index.html, game.js, and phaser.min.js into the 'game_files' directory in Colab.

--- Starting Flask Server (Foreground - Blocking) ---
Flask is now running and listening on port 8880.
Logs should appear directly below as requests come in.
!!! The Colab cell will appear 'busy'. To stop the server, you MUST interrupt the kernel !!!
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8880
 * Running on http://172.28.0.12:8880
INFO:werkzeug:Press CTRL+C to quit


## EXTRA SHIT

In [ ]:
'''
from flask import Flask, request, jsonify, send_from_directory
from flask_cors import CORS
import threading
import os
from pyngrok import ngrok # Import ngrok

# --- Flask App Setup ---
app = Flask(__name__, static_folder='game_files') # Point static folder
CORS(app) # Enable Cross-Origin Resource Sharing

# --- Game Logic Placeholder (will be refined) ---
# Store game state server-side IF needed, or manage fully client-side and send state with each request
# For this POC, let's assume the client (Phaser) manages state and sends it.

# --- LLM Interaction Function ---
def get_llm_decision(game_state, available_actions, user_instruction):
    if not llm_model:
         return {"error": "LLM model not initialized."}

    # --- Crucial: Prompt Engineering ---
    # Provide context, rules, state, available actions, and the user's goal.
    # Ask for the output in a specific, parseable format (like JSON).
    prompt = f"""You are playing a game. Respond with ONLY the chosen action in the format specified.

    Game: Tic-Tac-Toe
    Your Mark: O (You are playing as O)
    Player Mark: X

    Current Board State:
    {json.dumps(game_state['board'], indent=2)}
    (Empty strings "" represent empty cells)

    Available Actions (Choose one by coordinates row, col):
    {json.dumps(available_actions, indent=2)}

    User Instruction: "{user_instruction}"

    Game Status: {game_state['status']}
    Current Turn: {game_state['turn']}

    Based on the user instruction and the current state, choose the best action from the available actions for player 'O'.
    Respond ONLY with the chosen action as a JSON object like this: {{"action": "place O at (row, col)"}} or like this if you need coordinates: {{"row": r, "col": c}}
    Example response for placing O in the top-left: {{"row": 0, "col": 0}}
    """

    print("--- Sending Prompt to LLM ---")
    # print(prompt) # Uncomment to debug the prompt

    try:
        response = llm_model.generate_content(prompt)
        print("--- Received Response from LLM ---")
        print(response.text)

        # --- Parse the LLM Response ---
        # This is critical and might need robust error handling
        try:
            # Try parsing directly if LLM gives perfect JSON (unlikely)
            # action_data = json.loads(response.text.strip())

            # More robust: Search for JSON-like structure or extract coords
            # Example naive extraction (adapt based on LLM output format):
            import re
            match = re.search(r'{\s*"row":\s*(\d+),\s*"col":\s*(\d+)\s*}', response.text)
            if match:
                row, col = int(match.group(1)), int(match.group(2))
                # Validate if this action is actually in available_actions
                if {"row": row, "col": col} in available_actions:
                    print(f"LLM chose valid action: row={row}, col={col}")
                    return {"row": row, "col": col}
                else:
                     print(f"LLM chose an INVALID action ({row},{col}), not in available actions.")
                     return {"error": f"LLM chose an invalid action: ({row},{col})"}

            # Fallback or different parsing if needed
            return {"error": "Could not parse valid action from LLM response.", "raw_response": response.text}

        except json.JSONDecodeError:
             print(f"LLM response was not valid JSON: {response.text}")
             return {"error": "LLM response was not valid JSON.", "raw_response": response.text}
        except Exception as e:
             print(f"Error parsing LLM response: {e}")
             return {"error": f"Error parsing LLM response: {e}", "raw_response": response.text}

    except Exception as e:
        print(f"Error calling LLM API: {e}")
        return {"error": f"Error calling LLM API: {e}"}

# --- API Endpoint ---
@app.route('/api/llm_move', methods=['POST'])
def handle_llm_move():
    data = request.json
    game_state = data.get('gameState')
    available_actions = data.get('availableActions') # JS needs to calculate these!
    user_instruction = data.get('instruction')

    if not game_state or not available_actions or user_instruction is None:
        return jsonify({"error": "Missing gameState, availableActions, or instruction"}), 400

    # Basic validation (more needed in a real app)
    if game_state.get('turn') != 'O': # Assuming LLM plays 'O'
         return jsonify({"error": "Not LLM's turn"}), 400
    if game_state.get('status') != 'playing':
         return jsonify({"error": "Game is not active"}), 400

    decision = get_llm_decision(game_state, available_actions, user_instruction)
    return jsonify(decision)

# --- Static File Serving ---
# Create a directory in Colab's file system to hold your game files
if not os.path.exists('game_files'):
    os.makedirs('game_files')

# You need to UPLOAD your index.html, game.js, phaser.min.js, etc.
# into this 'game_files' directory using Colab's file browser (folder icon on left).

@app.route('/')
def index():
    # Serves index.html from the 'game_files' directory
    return send_from_directory('game_files', 'index.html')

@app.route('/<path:filename>')
def serve_static(filename):
    # Serves other files (game.js, phaser.min.js, assets)
    return send_from_directory('game_files', filename)

# --- Function to Run Flask App ---
def run_flask():
  # Needs to run on port 8080 for some Colab environments, or choose another
  # Ngrok will tunnel to this port
  print("Starting Flask server...")
  app.run(host='0.0.0.0', port=8880) # Run on all interfaces, port 8080

# --- Start ngrok tunnel ---
def start_ngrok():
    try:
        # Terminate existing tunnels if any
        ngrok.kill()
        # Get ngrok auth token from Colab secrets if you have one (recommended for stable URLs)
        try:
             ngrok_auth_token = userdata.get('NGROK_AUTH_TOKEN')
             ngrok.set_auth_token(ngrok_auth_token)
             print("Ngrok auth token set.")
        except userdata.SecretNotFoundError:
             print("INFO: NGROK_AUTH_TOKEN secret not found. Using ngrok without auth token (temporary URL).")
        except Exception as e:
             print(f"Error setting ngrok auth token: {e}")

        # Start an HTTP tunnel on the same port Flask is running on
        public_url = ngrok.connect(8880, "http")
        print(f" * ngrok tunnel available at: {public_url}")
        return public_url
    except Exception as e:
        print(f"Error starting ngrok: {e}")
        return None

# --- Main Execution Logic ---
# Start Flask in a separate thread so it doesn't block Colab execution
flask_thread = threading.Thread(target=run_flask, daemon=True)
flask_thread.start()

# Start ngrok and print the public URL
public_url = start_ngrok()

print("\n--- Server Setup Complete ---")
if public_url:
    print(f"Access your game POC at: {public_url}")
else:
    print("Failed to start ngrok tunnel. Server might be running but not accessible publicly.")
print("Flask server is running in the background.")
print("LLM endpoint is available at /api/llm_move (relative to the ngrok URL)")
print("Make sure you have uploaded your index.html, game.js, and phaser.min.js into the 'game_files' directory in Colab.")
# Keep the Colab cell running...
'''

In [ ]:
!curl http://127.0.0.1:8080/

In [ ]:
!lsof -i :8080

In [ ]:
!kill -9 6